In [1]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 259603, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
Receiving objects: 100% (259603/259603), 269.24 MiB | 3.00 MiB/s, done.
remote: Total 259603 (delta 14), reused 9 (delta 7), pack-reused 259576 (from 2)
Resolving deltas: 100% (191680/191680), done.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file
file_path = "/Users/srinivaskalyan/Documents/deployment/synthetic_data.csv"  # Update with the actual path
df = pd.read_csv(file_path)

# Convert labels to integers
label_mapping = {"Classification": 0, "Segmentation": 1, "Both": 2}
df["Label"] = df["Label"].map(label_mapping)

# Split dataset into train (80%) and validation (20%)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["Label"])

# Save the partitioned datasets
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)

print("Dataset partitioned successfully!")


Dataset partitioned successfully!


In [ ]:
!python "/Users/srinivaskalyan/Documents/deployment/transformers/examples/pytorch/text-classification/run_classification.py" \
    --model_name_or_path "google-bert/bert-base-uncased" \
    --train_file "/Users/srinivaskalyan/Documents/deployment/train.csv" \
    --validation_file "/Users/srinivaskalyan/Documents/deployment/val.csv" \
    --text_column_name "Text Instruction" \
    --label_column_name "Label" \
    --shuffle_train_dataset \
    --metric_name accuracy \
    --do_train \
    --do_eval \
    --max_seq_length 512 \
    --per_device_train_batch_size 4 \
    --gradient_checkpointing \
    --fp16 \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir "/Users/srinivaskalyan/Documents/deployment" \


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Path to the trained model and tokenizer
model_path = "/Users/srinivaskalyan/Documents/deployment/checkpoint-1400"

# Load the tokenizer and model from the checkpoint
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# The sentence you want to classify
sentence = "Analyze the image and distinguish where the inflammation is."

# Preprocess the sentence
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Run the model to get predictions
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class label (the index of the highest logit)
predicted_class = torch.argmax(outputs.logits, dim=-1).item()

# Print the result
print(f"Predicted class label: {predicted_class}")
